# Pipeline for interpolation of Keyframes


## TODO


In [ ]:
# TODO : Add pipeline for keyframe generation here as well (IMP)
# DONE : Stitch the final video together after running interpolation on keyframe pairs
# DONE : Figure out why the kernel starts shaking crying throwing up when I run the interpolation code. (IMP)
# Basically it dies at the last iteration for some reason. If i move the pipe out to save the time for loading it for every interpolation, it dies at second iteration. My thinking is that it is related to the cache/pipe/scehular being deleted/emtpied at wrong time/or maybe just that memeory becomes full but honeslty the latter dont be making sense cuz of the variation in wehn it does as described above
# TODO : Compare across different models (LATER)
# TODO : Add a function for displaying graphs (LATER)
# TODO : Compare across settings. (LATER)
# TODO : Add a function for calculating the PSNR and SSIM or whatever similarity metric we decide to use (IMP)
# TODO : Add the requirements.txt for this notebook (LATER)
# TODO : Make the markdown cells more informative, add description of what each cell does (LATER)

# TODO : explore algorithmic interpolation
# TODO : Add frame generation to the pipeline as well
# TODO : Color smoothen/correct final video 



In [ ]:
# !pip freeze > our_requirements.txt

## Imports

This code uses the `diffusers-0-27-0` environment
<br>
<span style="color:red">**NOTE:** Define all imports and install/shell commands here</span>


In [ ]:
# %%capture
# add install or other terminal commands here
# %pip install numpy matplotlib opencv-python scikit-image scikit-video pillow
# %pip install tabulate
# %conda install -c conda-forge ffmpeg

In [ ]:
import cv2
import subprocess
import sys
from KeyFrameDetector.key_frame_detector import keyframeDetectionByChunks
from KeyFrameDetector.key_frame_detector import smartKeyframeDetection
from utils.extract_frames import strawman_frame_extraction
from utils.katna import num_frames_keyframe
from utils.perceptual_similarity import calculate_lpips_distance
from utils.extract_frames import process_and_extract_frames
import numpy as np
from moviepy.editor import concatenate_videoclips, VideoFileClip
from tabulate import tabulate
from svd.attn_ctrl.attention_control import (
    AttentionStore,
    register_temporal_self_attention_control,
    register_temporal_self_attention_flip_control,
)
from svd.custom_diffusers.schedulers.scheduling_euler_discrete import EulerDiscreteScheduler
from svd.custom_diffusers.pipelines.pipeline_frame_interpolation_with_noise_injection import FrameInterpolationWithNoiseInjectionPipeline
from diffusers import UNetSpatioTemporalConditionModel
from diffusers.utils import load_image, export_to_video
import time
import torch
import shutil
import os
from utils.ffmpeg_evaluator import FFmpegEvaluator

os.chdir("/root/VideoReconstruction")

## Pipeline

There are two functuions in this pipeline:

1. `interpolate_keyframes` - This function takes in 2 keyframes and interpolates the keyframes to get the intermediate keyframes.
2. `generate_video` - This function uses a list of keyframes and interpolates between every two consecutive keyframes to get a list of intermediate video segments which are then concatenated to get the final video.


### Interpolation

The `run_interpolation` function interpolates between two keyframes and saves the output video to the specifed path.

The  `generate_all_interpolations` function generates all the intermediate keyframes between every two consecutive keyframes in an input directory with all the keyframes in it saving all the output videos to the output path.

The `stitch_video_segments` function stitches the video segments to get the final video.

In [ ]:
def run_interpolation(checkpoint_dir, frame1_path, frame2_path, out_path, resize_specs, fps, pretrained_model_name_or_path, duration, num_frames, seed=42, num_inference_steps=50, weighted_average=False, noise_injection_steps=0, noise_injection_ratio=0.5, decode_chunk_size=8, device="cuda:0"):
    """
    Run key frame interpolation between two frames using a pretrained model and noise injection. It saves the interpolated video or gif to the specified output path.

    Args:
        checkpoint_dir (str): Directory containing the checkpoint for the fine-tuned UNet model.
        frame1_path (str): Path to the first frame image.
        frame2_path (str): Path to the second frame image.
        out_path (str): Path to save the output interpolated video or gif.
        resize_specs (tuple): Tuple specifying the resize dimensions (width, height) for the input frames.
        fps (int): Frames per second for the output video.
        pretrained_model_name_or_path (str): Path or name of the pretrained model.
        duration (int): Duration for which each frame will be displayed in the gif (in milliseconds).
        num_frames (int): Number of frames to interpolate between the two input frames.
        seed (int, optional): Random seed for reproducibility. Defaults to 42.
        num_inference_steps (int, optional): Number of inference steps for the interpolation. Defaults to 50.
        weighted_average (bool, optional): Whether to use weighted average during interpolation. Defaults to False. 
        True: Produces a video with a gradual shift from image1 to image2. This can give the effect of morphing or smooth interpolation between two images.
        False: Maintains an equal influence of both images across all frames. This results in a more consistent combination of features from both image1 and image2.

        noise_injection_steps (int, optional): Number of steps for noise injection. Defaults to 0.
        noise_injection_ratio (float, optional): Ratio of noise injection. Defaults to 0.5.
        decode_chunk_size (int, optional): Chunk size for decoding. Defaults to 8. controls how many frames are decoded at a time from the latent representations during the video generation process.


        device (str, optional): Device to run the interpolation on. Defaults to "cuda:0".

    Returns:
        None
    """
    # Load noise scheduler and pipeline
    noise_scheduler = EulerDiscreteScheduler.from_pretrained(
        pretrained_model_name_or_path, subfolder="scheduler")

    pipe = FrameInterpolationWithNoiseInjectionPipeline.from_pretrained(
        pretrained_model_name_or_path, scheduler=noise_scheduler, variant="fp16", torch_dtype=torch.float16)

    # Set up UNet model for fine-tuning and load state dicts

    ref_unet = pipe.ori_unet
    state_dict = pipe.unet.state_dict()

    # Compute delta weights
    finetuned_unet = UNetSpatioTemporalConditionModel.from_pretrained(
        checkpoint_dir, subfolder="unet", torch_dtype=torch.float16)

    ori_unet = UNetSpatioTemporalConditionModel.from_pretrained(
        pretrained_model_name_or_path, subfolder="unet", variant="fp16", torch_dtype=torch.float16)

    # Apply delta to state dict for specific layers
    finetuned_state_dict = finetuned_unet.state_dict()
    ori_state_dict = ori_unet.state_dict()
    for name, param in finetuned_state_dict.items():
        if "temporal_transformer_blocks.0.attn1.to_v" in name or "temporal_transformer_blocks.0.attn1.to_out.0" in name:
            delta_w = param - ori_state_dict[name]
            state_dict[name] = state_dict[name] + delta_w
    pipe.unet.load_state_dict(state_dict)

    # Setup attention controllers
    controller_ref = AttentionStore()
    register_temporal_self_attention_control(ref_unet, controller_ref)

    controller = AttentionStore()
    register_temporal_self_attention_flip_control(
        pipe.unet, controller, controller_ref)

    # Move pipeline to specified device
    pipe = pipe.to(device)

    # Set random seed
    generator = torch.Generator(device=device)
    if seed is not None:
        generator = generator.manual_seed(seed)

    # Load and resize frames
    frame1 = load_image(frame1_path).resize(resize_specs)
    frame2 = load_image(frame2_path).resize(resize_specs)

    # Perform inference
    # print(f"frame 1 path ", frame1_path)
    # print(f"frame 2 path ", frame2_path)

    timestamp_f1 = frame1_path.split("_")[1]
    timestamp_f2 = frame2_path.split("_")[1]
    # print(
    #     f"after splitting the list for f1 is {timestamp_f1} and for f2 is {timestamp_f2}")
    timestamp_f1 = timestamp_f1.split(".jpg")[0]
    timestamp_f2 = timestamp_f2.split(".jpg")[0]
    # print(f"timestamp_f1 : {timestamp_f1}")
    # print(f"timestamp_f2 : {timestamp_f2}")

    num_frames = int(
        np.round((float(timestamp_f2) - float(timestamp_f1)) * fps))

    if num_frames > 24:
        num_frames = 24
    elif num_frames < 2:
        num_frames = 2

    # print(f"num_frames : {num_frames}")

    frames = pipe(image1=frame1, image2=frame2, num_inference_steps=num_inference_steps, generator=generator, weighted_average=weighted_average,
                  noise_injection_steps=noise_injection_steps, noise_injection_ratio=noise_injection_ratio, num_frames=num_frames).frames[0]

    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    # duration = the time  for whicch each frame will be displayed in the gif
    if out_path.endswith(".gif"):
        print(f"Saving {len(frames)} frames to {out_path} as gif")
        frames[0].save(out_path, save_all=True,
                       append_images=frames[1:], duration=duration, loop=0)
    else:
        print(f"Saving {len(frames)} frames to {out_path} as video")
        export_to_video(frames, out_path, fps=fps)

    print(f"Interpolated video saved to {out_path}")

    # Free GPU memory after inference
    del controller, controller_ref, ori_unet, finetuned_unet
    torch.cuda.empty_cache()

In [ ]:
def generate_all_interpolations(checkpoint_dir, input_sub_dir, output_dir, model_name, num_frames, decode_chunk_size, extension=".gif", resize_specs=(1024, 576), fps=7, duration=142, seed=42, inference_steps=20, noise_injection_steps=2, noise_injection_ratio=0.5, device="cuda:0", video_name=None):
    """
    Generates interpolated videos from key frames in the input directory.

    Args:
        checkpoint_dir (str): Directory containing the model checkpoints.
        input_sub_dir (str): Directory containing the input frames.
        output_dir (str): Directory to save the output videos.
        model_name (str): Name of the pre-trained model to use for interpolation.
        num_frames (int): Number of frames to generate between each pair of key frames.
        decode_chunk_size (int): Size of the chunk to decode.
        extension (str, optional): Extension of the output video files. Defaults to ".gif".
        resize_specs (tuple, optional): Tuple specifying the width and height to resize the frames. Defaults to (1024, 576).
        fps (int, optional): Frames per second for the output video. Defaults to 7.
        duration (int, optional): Duration of the output video in seconds. Defaults to 142.
        seed (int, optional): Random seed for reproducibility. Defaults to 42.
        inference_steps (int, optional): Number of inference steps for the model. Defaults to 20.
        noise_injection_steps (int, optional): Number of steps to inject noise during inference. Defaults to 2.
        noise_injection_ratio (float, optional): Ratio of noise to inject during inference. Defaults to 0.5.
        device (str, optional): Device to run the model on. Defaults to "cuda:0".
        video_name (str, optional): Name of the video. If None, it will be derived from the input_sub_dir. Defaults to None.

    Returns:
        None
    """
    # video_name = os.path.basename(input_sub_dir)

    # print(f"Generating video for {video_name}")
    video_name = os.path.basename(input_sub_dir)

    intermediate_videos_dir = os.path.join(
        output_dir, f"interm_videos_{video_name}")
    os.makedirs(intermediate_videos_dir, exist_ok=True)

    frames = sorted([os.path.join(input_sub_dir, f) for f in os.listdir(
        input_sub_dir) if f.endswith((".png", ".jpeg", ".jpg"))])

    print(f"Found {len(frames)} frames in {input_sub_dir}")
    if len(frames) < 2:
        print(f"Skipping {video_name} as it has less than 2 frames")
        return

    print(f"Frames : {frames}")
    frames = sorted(frames, key=lambda x: float(
        x.split("_")[-1].replace(".jpg", "")))
    print(f"Frames : {frames}")
    for i in range(len(frames) - 1):
        frame1_path = frames[i]
        frame2_path = frames[i + 1]

        print(f"Interpolating between frames {frame1_path} and {frame2_path}")

        if "<s>" in frame1_path:
            print(f"Skipping {frame1_path} as its entrie bucket is already saved")
            continue
        
        segment_output_path = os.path.join(
            intermediate_videos_dir, f"segment_{i}{extension}")

        run_interpolation(
            checkpoint_dir=checkpoint_dir,
            frame1_path=frame1_path,
            frame2_path=frame2_path,
            out_path=segment_output_path,
            resize_specs=resize_specs,
            fps=fps,
            pretrained_model_name_or_path=model_name,
            duration=duration,
            seed=seed,
            num_inference_steps=inference_steps,
            noise_injection_ratio=noise_injection_ratio,
            noise_injection_steps=noise_injection_steps,
            num_frames=num_frames,
        )
        torch.cuda.empty_cache()
    torch.cuda.empty_cache()

In [ ]:
def stitch_video_segments(intermediate_videos_dir, output_path, fps):
    """
    Stitches together video segments from a specified directory into a single video file.

    Args:
        intermediate_videos_dir (str): The directory containing the video segments to be stitched together.
        output_path (str): The file path where the final stitched video will be saved.
        fps (int): The frames per second (fps) rate for the final video.

    Returns:
        None

    Prints:
        The number of segments found and a confirmation message when the final video is saved.
    """
    segment_paths = sorted([os.path.join(intermediate_videos_dir, f)
                           for f in os.listdir(intermediate_videos_dir)])

    print(f"Found {len(segment_paths)} segments to stitch.")
    clips = [VideoFileClip(segment).set_fps(fps) for segment in segment_paths]
    final_video = concatenate_videoclips(clips, method="compose")
    final_video.write_videofile(output_path, fps=fps)
    print(f"Final video saved to {output_path}")

    for clip in clips:
        clip.close()

### Keyframe Generation and Evaluation


In [ ]:
def get_directory_size(directory):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(directory):
        for filename in filenames:
            filepath = os.path.join(dirpath, filename)
            total_size += os.path.getsize(filepath)
    return total_size

### Keyframe size savings


This part of the code takes all the videos in input folder and generates keyframes for each video and stores them in the output folder. It also benchmarks the sizes of the original videos and videos from keyframes.


In [ ]:

input_dir = "/root/VideoReconstruction/input_resized"
output_dir_base = "/root/VideoReconstruction/outputs/bucketed_keyframes"
smaller_video_output_dir = "/root/VideoReconstruction/outputs/bucketed_keyframes_video"

os.makedirs(output_dir_base, exist_ok=True)
os.makedirs(smaller_video_output_dir, exist_ok=True)

videos = os.listdir(input_dir)

summary_data = []

evaluator = FFmpegEvaluator()

for video in videos:
    video_path = os.path.join(input_dir, video)
    video_name = os.path.splitext(video)[0]
    output_dir = os.path.join(output_dir_base, video_name)

    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    cap.release()

    video_size_in_bytes = os.path.getsize(video_path)
    video_size_in_megabytes = video_size_in_bytes / (1024 * 1024)

    keyframes = smartKeyframeDetection(video_path, output_dir, threshold = 0.3, bucket_size_in_frames = 80, minimum_frames_between=20, maximum_frames_between = 30)
    # keyframes = keyframeDetectionByChunks(video_path, output_dir, number_frames_per_bucket=60, top_k=5, verbose=False, minimum_frames_between=20, maximum_frames_between=30)
    keyframe_count = len(keyframes)

    keyframes_dir_size_in_bytes = get_directory_size(output_dir)
    keyframes_dir_size_in_megabytes = keyframes_dir_size_in_bytes / (1024 * 1024)

    output_dir = f"{output_dir}/keyFrames"
    compressed_size, savings_from_original_video, savings_from_keyframes_video = evaluator.evaluate_smaller_video(
        video_path, smaller_video_output_dir, output_dir)

    summary_data.append(
        [
            video,
            f"{video_size_in_megabytes:.2f} MB",
            total_frames,
            f"{fps:.2f} FPS",
            keyframe_count,
            f"{keyframes_dir_size_in_megabytes:.2f} MB",
            f"{compressed_size/(1024*1024):.2f} MB",
            f"{savings_from_original_video:.2f} %",
            f"{savings_from_keyframes_video:.2f} %"
        ]
    )

headers = [
    "Video Name",
    "Original Size",
    "Total Frames",
    "FPS",
    "Total Keyframes",
    "Keyframes Size",
    "Compressed Video Size",
    "Savings Original Video",
    "Savings Keyframes Video"
]
print(tabulate(summary_data, headers=headers, tablefmt="grid"))

### Run the pipeline


#### Constants


In [ ]:
input_dir = "/root/VideoReconstruction/outputs/bucketedkeyframes/beach"
# output_dir = "/VideoReconstruction/interim"
output_dir = "/root/VideoReconstruction/interim/keyFrames"
# ============ ONLY THE FOLLOWING TWO PATHS ARE BEING USED IN *INTERPOLATION* ============
# INPUT_DIR is the directory from which the keyframes will be read
INPUT_DIR = "/root/VideoReconstruction/outputs/bucketedkeyframes"
input_subdir_name = "beach"
# OUT_DIR + interim_videos_<vidname> is the directory where the interpolated segments will be saved
# OUT_DIR + <vidname>_interpolate is the filename of the final stitched video
OUT_DIR = "/root/VideoReconstruction/our_results"
os.makedirs(OUT_DIR, exist_ok=True)
threshold = 0.3


# Model options and selection. maybe we cam add more models later for this
models_to_try = ["stabilityai/stable-video-diffusion-img2vid-xt",
                 "stabilityai/stable-video-diffusion-img2vid-xt-1-1"]
MODEL_NAME = models_to_try[0]

# Noise injection parameters
noise_injection_steps = 2
noise_injection_ratio = 0.5

# All Directory paths are defined here
# CHECKPOINT_DIR = "/home/iml2/interpolation/svd_keyframe_interpolation/checkpoints/svd_reverse_motion_with_attnflip/svd_reverse_motion_with_attnflip/unet" #path for iml2
CHECKPOINT_DIR = "/root/VideoReconstruction/svd/checkpoints/svd_reverse_motion_with_attnflip/svd_reverse_motion_with_attnflip/unet"  # path for docker image
# if this is .gif, the output will be a gif otherwise it will be a video
interpolation_extension = ".mp4"
final_extension = ".mp4"

# Resize specifications, frame rate, and duration etc
resize_specs = (1024, 576)
fps = 15
duration = 142  # the duration for which each frame is displayed in the vid
inference_steps = 20
seed = 42
decode_chunk_size = 6
num_frames = 7


# INPUT_DIR = "/home/iml2/manframes"
# INPUT_DIR = "/VideoReconstruction/input"
# input_subdir_name = "horses"
input_subdir_path = os.path.join(INPUT_DIR, input_subdir_name)

# # the katana measure to get frames but they come without timestamps
# num_frames_keyframe(video_path, output_dir, num_frames)


# ===========================
# Display Configuration
# ===========================

settings = [
    ["Model", MODEL_NAME],
    ["Output Directory", OUT_DIR],
    ["Input Directory", input_subdir_name],
    ["Resize Specifications", f"{resize_specs[0]} x {resize_specs[1]}"],
    ["FPS", fps],
    ["Frame Duration (ms)", duration],
    ["Inference Steps", inference_steps],
    ["Random Seed", seed],
    ["Saving interpolated video as", "gif file" if interpolation_extension ==
        ".gif" else "mp4 video file"],
    ["Number of Frames", num_frames],
]

print("\nSettings for Image-to-Video Interpolation:")
print(tabulate(settings, headers=["Parameter", "Value"], tablefmt="grid"))

Here, we loop through all the videos in the input folder, generate keyframes for them and then interpolate between the keyframes to get the final video. This is done for all the videos in the input directory.

In [ ]:
# Loop over each file in the input directory
for filename in os.listdir("./input_videos/"):
    start_time = time.time()
    # Get the base name without the extension 
    video_name = os.path.splitext(filename)[0]
    # Define the full path to the video file
    video_path = os.path.join(input_dir, filename)
    try:
        # print("input_sub_dir: ", os.path.join(INPUT_DIR, video_name))
        # print("OUT_DIR: ", OUT_DIR)
        # continue
        generate_all_interpolations(checkpoint_dir=CHECKPOINT_DIR, input_sub_dir=os.path.join(INPUT_DIR, video_name), output_dir=OUT_DIR, model_name=MODEL_NAME, resize_specs=resize_specs, fps=fps, duration=duration, extension=interpolation_extension,
                                    seed=seed, inference_steps=inference_steps, noise_injection_steps=noise_injection_steps, noise_injection_ratio=noise_injection_ratio, decode_chunk_size=decode_chunk_size, num_frames=num_frames, video_name=video_name)
    except Exception as e:
        print("Error while generate_all_interpolations")
        print(f"Error: {e}")
        # i am doing this cuz it is annoying to get out of memory erros every time there is an error. so we will kill the process after it is done/error thrown
        print("Killing the kernel processes")
        env_name = os.path.basename(sys.prefix)
        subprocess.call(
            f"nvidia-smi | grep {env_name} | awk '{{print $5}}' | xargs -I {{}} kill -9 {{}}",
            shell=True
        )
        raise e
    
    try:
        stitch_video_segments(intermediate_videos_dir=os.path.join(OUT_DIR, f"interm_videos_{video_name}"), output_path=os.path.join(
            OUT_DIR, f"{video_name}_interpolated{final_extension}"), fps=fps)
    except Exception as e:
        print(f"Error: {e}")
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Elapsed time for {video_name}: {elapsed_time:.2f} seconds")

In [ ]:
# try:
#     generate_all_interpolations(checkpoint_dir=CHECKPOINT_DIR, input_sub_dir=input_subdir_path, output_dir=OUT_DIR, model_name=MODEL_NAME, resize_specs=resize_specs, fps=fps, duration=duration, extension=interpolation_extension,
#                     seed=seed, inference_steps=inference_steps, noise_injection_steps=noise_injection_steps, noise_injection_ratio=noise_injection_ratio, decode_chunk_size=decode_chunk_size, num_frames=num_frames)
# except Exception as e:
#     print("Error while generate_all_interpolations")
#     print(f"Error: {e}")
#     # i am doing this cuz it is annoying to get out of memory erros every time there is an error. so we will kill the process after it is done/error thrown
#     print("Killing the kernel processes")
#     env_name = os.path.basename(sys.prefix)
#     subprocess.call(
#     f"nvidia-smi | grep {env_name} | awk '{{print $5}}' | xargs -I {{}} kill -9 {{}}",
#     shell=True
#     )


In [ ]:

# try:
#     stitch_video_segments(intermediate_videos_dir=os.path.join(OUT_DIR, f"interm_videos_{input_subdir_name}"), output_path=os.path.join(
#         OUT_DIR, f"{input_subdir_name}_interpolated{final_extension}"), fps=fps)
# except Exception as e:
#     print(f"Error: {e}")
# end_time = time.time()
# elapsed_time = end_time - start_time
# print(f"Elapsed time for {video_name}: {elapsed_time:.2f} seconds")

In [ ]:
!nvidia-smi | grep $(conda env list | grep '*' | awk '{print $1}') | awk '{print $5}' | xargs -I {} kill -9 {}

## Evaluation

We evaluate the generated video against the original video using several methods.


In [ ]:
# Adjust the paths and other hyper parameters of the two videos to compare against each other.
generated_video = "office_dude/1.mp4"
# generated_video = "our_results_strawman/car_drive_interpolated.mp4"
original_video = "office_dude/2.mp4"
# original_video = "input/car_drive.mp4"
frame_width = 1024
frame_height = 576
max_frames = 300

# make directories for storing the frames of both of these videos if they dont exist already
frames_dir = "frames"
if os.path.exists(frames_dir):
    shutil.rmtree(frames_dir)
generated_video_frame_dir = os.path.join(frames_dir, "input")
original_video_frame_dir = os.path.join(frames_dir, "output")
if not os.path.exists(frames_dir):
    os.makedirs(frames_dir)
if not os.path.exists(generated_video_frame_dir):
    os.makedirs(generated_video_frame_dir)
if not os.path.exists(original_video_frame_dir):
    os.makedirs(original_video_frame_dir)

original_video_dir = "/VideoReconstruction/input"
generated_video_dir = "/VideoReconstruction/our_results_strawman"

# # get file basename form original video dir
# for file in os.listdir(original_video_dir):
#     if file.endswith(".mp4"):
#         filename = os.path.splitext(file)[0]
#         print(f"Processing {filename}")
#         generated_video = f"{generated_video_dir}/{filename}_interpolated.mp4"
#         original_video = f"{original_video_dir}/{filename}.mp4"
#         # break
#         # break both the videos down into frames
process_and_extract_frames(input_video_path=generated_video, frames_output_dir=generated_video_frame_dir,
                           new_width=frame_width, new_height=frame_height, max_frames=max_frames)

process_and_extract_frames(input_video_path=original_video, frames_output_dir=original_video_frame_dir,
                           new_width=frame_width, new_height=frame_height, max_frames=max_frames)
# Pass it the directories with the frames of original and generated video to retrieve the perceptual loss across each frame
calculate_lpips_distance(dir0=generated_video_frame_dir,
                         dir1=original_video_frame_dir, use_gpu=True)